In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
from tqdm import tqdm # Displays a progress bar
from PIL import Image as PImage
from pathlib import Path

import csv
import os
import glob
import cv2
import shutil
import math
import h5py
import random

import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F   
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, Subset, DataLoader, random_split, ConcatDataset
import torchvision.models as models

%cd /content
!gdown https://drive.google.com/uc?id=1Lqk7z7iVPG7vRl5-wcY0GPjjYwAy7Kfy
#https://drive.google.com/file/d/1Lqk7z7iVPG7vRl5-wcY0GPjjYwAy7Kfy/view
!unzip /content/DATA

In [ ]:
params = {
    "batchSize": 32,
    "epochs": 16,
    "learningRate": 0.001,
    "frozenChildren": 9,

}
folders = ['Covid', 'Healthy', 'Others']
newfolders = []

image_label = []

max_length = []

averageAccuracy1 = []
averageAccuracy2 = []
  
%cd /content/496data/
origDir = os.getcwd()
allDir = os.path.join(origDir, 'AllImages')
if os.path.isdir(allDir):
    shutil.rmtree(allDir)
os.mkdir(allDir)

In [ ]:
def combine_images(folderType):
    thisDir = os.path.join(origDir, folderType)
    newDir = os.path.join(origDir, 'new'+folderType)
    
    if os.path.isdir(newDir):
        shutil.rmtree(newDir)
    os.mkdir(newDir)
    newfolders.append(newDir)
    
    count = 0
    
    for patient in os.listdir(thisDir):
        patientPath = os.path.join(thisDir, patient)
        if os.path.isdir(patientPath):
            for file in glob.glob(os.path.join(patientPath, '*.png')):
                count+=1
                imagePath = os.path.join(patientPath, file)
                img = cv2.imread(imagePath)
                img = cv2.resize(img,  (224,224), interpolation=cv2.INTER_CUBIC)
                newPath = os.path.join(newDir, str(count) + '.png')
                cv2.imwrite(newPath, img)
                # shutil.copy(imagePath, newPath)
                
    
for folder in folders:
    combine_images(folder)
    
    
def combine_images2(folderType):
    count = 0
    label = -1
    
    for folder in folderType:
        label+=1
        for image in os.listdir(folder):
            count+=1
            max_length.append(count)
            imagePath = os.path.join(folder, image)
            newPath = os.path.join(allDir, str(count) + '.png')
            shutil.copy(imagePath, newPath)
            img = cv2.imread(newPath)
            image_label.append([newPath, label, count])
            img = cv2.flip(img, 1)
            count+=1
            max_length.append(count)
            newPath3 = os.path.join(allDir, str(count) + '.png')
            cv2.imwrite(newPath3, img)
            image_label.append([newPath3, label, count])
            for i in range(3):
                img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                count+=1
                max_length.append(count)
                newPath2 = os.path.join(allDir, str(count) + '.png')
                cv2.imwrite(newPath2, img)
                image_label.append([newPath2, label, count])
            
combine_images2(newfolders)
print("done")
print(len(max_length))

done
20895


In [ ]:
labelsEveryFolder = []
for i in range(5):
    temp = []
    labelsEveryFolder.append(temp)
sample = []
def create_random_folds(allimages):
    randFolder = 0
    randArray = []
    for i in range(5):
        randPath = os.path.join(origDir, 'Random' + str(i))
        if os.path.isdir(randPath):
            shutil.rmtree(randPath)
        os.mkdir(randPath)
        with open(os.path.join(randPath, "label.csv"), 'w+') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['origpath', 'label', 'directory'])
            pass
        randArray.append(randPath)
    random.shuffle(allimages)
    max_num = math.ceil(0.2*len(max_length))
    count = 0
    for row in allimages:
        imagePath = row[0]
        if count < max_num:
            temp = []
            temp.append(imagePath)
            temp.append(row[1])
            path = os.path.join(origDir, 'Random' + str(randFolder))
            shutil.copy(imagePath, path)
            temp.append(path)
            sample.append(temp)
            labelsEveryFolder[randFolder].append(temp)
            with open(os.path.join(path, "label.csv"), 'a') as csvfile:
                csvwriter = csv.writer(csvfile, delimiter=',', lineterminator='\r') 
                csvwriter.writerow(temp)
            temp = []
            count+=1
        else:
            randFolder+=1
            count = 0
    return randArray
datasetfolders = create_random_folds(image_label)
# print(datasetfolders)
# print(labelsEveryFolder)


In [ ]:
class DataSet:

    def __init__(self, root):
    
        self.ROOT = root
        self.images, self.mean, self.std = self.read_images(self.ROOT)
        self.labels = self.read_labels(self.ROOT)
        # Data normalization
        self.transform = transforms.Compose([
#         transforms.Grayscale(num_output_channels=1), # Convert image to grayscale
            transforms.ToTensor(), # Transform from [0,255] uint8 to [0,1] float
            transforms.Normalize((self.mean,), (self.std,))# TODO: Normalize to zero mean and unit variance with appropriate parameters
        ])
        
        

    def __len__(self):
    # Return number of points in the dataset

        return len(self.images)

    def __getitem__(self, idx):
    # Here we have to return the item requested by `idx`. The PyTorch DataLoader class will use this method to make an iterable for training/validation loop.

        img = self.transform(self.images[idx])
        img = np.concatenate((img,img,img), 0)
        label = self.labels[idx]

        return img, label
    
    def read_images(self, path:str) -> list:
        imgs = []
        mean = 0
        std = 0
        count = 0
        for filename in sorted([f for f in os.listdir(path) if f.endswith(".png")], key=lambda f : int(f[:-4])): 
            img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_UNCHANGED)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            imgs.append(img)
            mean += np.mean(img)
            std += np.var(img)
            count += 1
        mean /= count*255
        std = math.sqrt(std / (count)) / 255
        print(path)
        return imgs, mean, std
    
    def read_labels(self, path:str) -> list:
        labels = pd.read_csv(os.path.join(path, 'label.csv'))
        print(labels)
        return labels.label

# Load the dataset and train and test splits
print("Loading datasets...")
print("Done!")
print("Creating datasets")
split = []
everything = []
for i in range(5):
    curr = os.path.join(origDir, 'Random' + str(i))
    everything.append(DataSet(curr))
    split.append(DataSet(curr))
print("Done done")
print(everything)

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: [Transfer learning with pre-trained ResNet-50] 1) Define how many first layers of convolutoinal neural network (CNN) feature extractor in ResNet-50 to be "frozen" and 2) design your own fully-connected network (FCN) classifier.
        # 1) You will only refine last several layers of CNN feature extractor in ResNet-50 that mainly relate to high-level vision task. Determine how many first layers of ResNet-50 should be frozen to achieve best performances. Commented codes below will help you understand the architecture, i.e., "children", of ResNet-50.
        # 2) Design your own FCN classifier. Here I provide a sample of two-layer FCN.
        # Refer to PyTorch documentations of torch.nn to pick your layers. (https://pytorch.org/docs/stable/nn.html)
        # Some common Choices are: Linear, ReLU, Dropout, MaxPool2d, AvgPool2d
        # If you have many layers, consider using nn.Sequential() to simplify your code
        
        # Load pretrained ResNet-50
        self.model_resnet = models.resnet50(pretrained=True)
        
        # The code below can show children of ResNet-50
        # child_counter = 0
        # for child in self.model_resnet.children():
        #    print(" child", child_counter, "is -")
        #    print(child)
        #    child_counter += 1
        
            
        # TODO: Determine how many first layers of ResNet-50 to freeze
        #first iteration freeze all children
        child_counter = 0
        for child in self.model_resnet.children():
            if child_counter < params['frozenChildren']:
                for param in child.parameters():
                    param.requires_grad = False
            elif child_counter == params['frozenChildren']:
                children_of_child_counter = 0
                for children_of_child in child.children():
                    # if children_of_child_counter < ???:
                    #     for param in children_of_child.parameters():
                    #         para.requires_grad = False
                    # else:
                    children_of_child_counter += 1
            else:
                print("child ",child_counter," was not frozen")
            child_counter += 1
            
        #first iteration start with classifier relu
        
        # Set ResNet-50's FCN as an identity mapping
        num_fc_in = self.model_resnet.fc.in_features
        self.model_resnet.fc = nn.Identity()
    

        self.fc1 = nn.Linear(num_fc_in, 256, bias = False)
        self.b1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128, bias = False)
        self.b2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64, bias = False)
        self.b3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 3, bias = False)
        
    def forward(self,x):
        # TODO: Design your own network, implement forward pass here
        
        relu = nn.ReLU() # No need to define self.relu because it contains no parameters
        gelu = nn.GELU()
        
        with torch.no_grad():
            features = self.model_resnet(x)
        
        
        #maybe add batch normalizations
        x = self.fc1(features) # Activation are flattened before being passed to the fully connected layers
        x = self.b1(x)
        x = relu(x)

        x = self.fc2(x)
        x = self.b2(x)
        x = relu(x)

        x = self.fc3(x)
        x = self.b3(x)
        x = relu(x)
        x = self.fc4(x)
        
        # The loss layer will be applied outside Network class
        return x


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
model= Network().to(device)
criterion = nn.CrossEntropyLoss() # Specify the loss layer (note: CrossEntropyLoss already includes LogSoftMax())
# # TODO: Modify the line below, experiment with different optimizers and parameters (such as learning rate)
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=params['learningRate'])
num_epochs = params['epochs'] # TODO: Choose an appropriate number of training epochs

In [ ]:
avgTraining = []
avgTesting = []

plotTraining = []
plotTesting = []

def train(model, loader, test, num_epoch, optimizer): # Train the model
    print("Start training...")
    model.train() # Set the model to training mode
    trainingLoss = []
    testingLoss = []
    for i in range(num_epoch):
        running_loss = []
        testing_loss = []
        for batch, label in tqdm(loader, position=0, leave=True, disable=True):
            batch = batch.to(device)
            label = label.to(device)
            optimizer.zero_grad() # Clear gradients from the previous iteration
            pred = model(batch) # This will call Network.forward() that you implement
            loss = criterion(pred, label) # Calculate the loss
            running_loss.append(loss.item())
            loss.backward() # Backprop gradients to all tensors in the network
            optimizer.step() # Update trainable weights
        print("Epoch {} training loss:{}".format(i+1,np.mean(running_loss))) # Print the average loss for this epoch
        with torch.no_grad():
            for batch, label in tqdm(test, disable=True):
                batch = batch.to(device)
                label = label.to(device)
                pred = model(batch)
                loss = criterion(pred, label)
                testing_loss.append(loss.item()) 
        print("Epoch {} testing loss:{}".format(i+1,np.mean(testing_loss))) 
        trainingLoss.append(np.mean(running_loss))
        testingLoss.append(np.mean(testing_loss))  
    
    print("Done!")
    avgTraining.append(trainingLoss)
    avgTesting.append(testingLoss)
    plt.plot(trainingLoss, color='blue', label="train")
    plt.plot(testingLoss, color='red', label="test")
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend()
    plt.show()

def evaluate(model, loader): # Evaluate accuracy on validation / test set
    model.eval() # Set the model to evaluation mode
    correct = 0
    with torch.no_grad(): # Do not calculate grident to speed up computation
        for batch, label in tqdm(loader):
            batch = batch.to(device)
            #set batch size
            label = label.to(device)
            pred = model(batch)
            correct += (torch.argmax(pred,dim=1)==label).sum().item()
    acc = correct/len(loader.dataset)
    print("Evaluation accuracy: {}".format(acc))

    return acc
    
# train(model, trainloader, num_epochs)
# print("Evaluate on test set")
# evaluate(model, testloader)

In [ ]:
x = [0,1,2,3,4]
testIndex = []
trainIndex = []


print("Begin training...")

for i in range(5):
    model = Network().to(device)
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=params['learningRate']) # Specify optimizer and assign trainable parameters to it, weight_decay is L2 regularization strength (default: lr=1e-2, weight_decay=1e-4)
    testIndex.append(x[i])
    x.remove(i)
    for j in range(4):
        trainIndex.insert(j, x[j])
    x.insert(i, i)
    batchTrain = DataLoader(ConcatDataset([everything[trainIndex[0]], everything[trainIndex[1]], 
                                           everything[trainIndex[2]], everything[trainIndex[3]]]),
                         batch_size = params['batchSize'],
                         shuffle = True, drop_last = True)
    batchTest = DataLoader(everything[testIndex[0]], batch_size = params['batchSize'], 
                        shuffle = True, drop_last = True)
    testIndex = []
    trainIndex = []
    training = train(model, batchTrain, batchTest, num_epochs, optimizer)
    averageAccuracy1.append(evaluate(model, batchTest))
    averageAccuracy2.append(evaluate(model, batchTrain))
    print(f'Testing accuracy for batch {i}: {evaluate(model, batchTest)}')
    del model

print(avgTraining)
print(avgTesting)
for x in range(params['epochs']):
  tempArray1 = []
  tempArray2 = []
  for y in range(5):
    tempArray1.append(avgTraining[y][x])
    tempArray2.append(avgTesting[y][x])
  plotTraining.append(np.mean(tempArray1))
  plotTesting.append(np.mean(tempArray2))
  print(tempArray1)
  print(tempArray2)

plt.plot(plotTraining, color='blue', label="training avg")
plt.plot(plotTesting, color='red', label="testing avg")
plt.ylabel('loss avg')
plt.xlabel('epoch')
plt.legend()
plt.show()

print(f'Average Training accuracy: {np.mean(averageAccuracy2)}')
print(f'Average Testing accuracy: {np.mean(averageAccuracy1)}')
